# Timeseries classification with a Transformer model

**Author:** [Theodoros Ntakouris](https://github.com/ntakouris)<br>
**Date created:** 2021/06/25<br>
**Last modified:** 2021/08/05<br>
**Description:** This notebook demonstrates how to do timeseries classification using a Transformer model.

## Introduction

This is the Transformer architecture from
[Attention Is All You Need](https://arxiv.org/abs/1706.03762),
applied to timeseries instead of natural language.

This example requires TensorFlow 2.4 or higher.

## Load the dataset

We are going to use the same dataset and preprocessing as the
[TimeSeries Classification from Scratch](https://keras.io/examples/timeseries/timeseries_classification_from_scratch)
example.

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test (1).csv
Saving train.csv to train (1).csv
Saving valid.csv to valid (1).csv


In [1]:
import numpy as np
import pandas as pd
import keras
from keras import layers


def readucr(filename):
    data = pd.read_csv(filename, header=None)
    data = data.values
    y = data[:, -1]
    x = data[:, :-1]
    return x, y.astype(int)


x_train, y_train = readucr("train.csv")
x_valid, y_valid = readucr("valid.csv")
x_train = np.concatenate([x_train, x_valid], axis = 0)
y_train = np.concatenate([y_train, y_valid], axis = 0)
x_test, y_test = readucr("test.csv")

x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

n_classes = len(np.unique(y_train))

idx = np.random.permutation(len(x_train))
x_train = x_train[idx]
y_train = y_train[idx]

y_train[y_train == -1] = 0
y_test[y_test == -1] = 0

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
def Normalization_on_data(input_f):
    '''
    input_f: N, C, L (torch.tensor)
    Output: N, C, L (normalized)
    '''
    miu = np.mean(input_f, axis=1, keepdims=True)    # shape: [N, C, 1]
    input_f = input_f - miu
    sigma = input_f.std(axis=1, keepdims=True)          # shape: [N, C, 1]
    input_f = input_f / sigma
    return input_f

x_train = Normalization_on_data(x_train)
x_test = Normalization_on_data(x_test)

## Build the model

Our model processes a tensor of shape `(batch size, sequence length, features)`,
where `sequence length` is the number of time steps and `features` is each input
timeseries.

You can replace your classification RNN layers with this one: the
inputs are fully compatible!

We include residual connections, layer normalization, and dropout.
The resulting layer can be stacked multiple times.

The projection layers are implemented through `keras.layers.Conv1D`.

In [ ]:
# This implementation applies Layer Normalization before the residual connection
# to improve training stability by producing better-behaved gradients and often
# eliminating the need for learning rate warm-up.

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Attention and Normalization
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = layers.Dropout(dropout)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    return x + res


The main part of our model is now complete. We can stack multiple of those
`transformer_encoder` blocks and we can also proceed to add the final
Multi-Layer Perceptron classification head. Apart from a stack of `Dense`
layers, we need to reduce the output tensor of the `TransformerEncoder` part of
our model down to a vector of features for each data point in the current
batch. A common way to achieve this is to use a pooling layer. For
this example, a `GlobalAveragePooling1D` layer is sufficient.

In [ ]:

def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_last")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)


## Train and evaluate

In [ ]:
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=128,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[64],
    mlp_dropout=0.1,
    dropout=0.05,
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["sparse_categorical_accuracy"],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=150,
    batch_size=64,
    callbacks=callbacks,
)

model.evaluate(x_test, y_test, verbose=1)

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8       │ (None, 426, 1)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 426, 1)    │      3,585 │ input_layer_8[0]… │
│ (MultiHeadAttentio… │                   │            │ input_layer_8[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_87          │ (None, 426, 1)    │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 426, 1)    │          2 │ dropout_87[0][0]  │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_52 (Add)        │ (None, 426, 1)    │          0 │ layer_normalizat… │
│                     │                   │            │ input_layer_8[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_52 (Conv1D)  │ (None, 426, 4)    │          8 │ add_52[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_88          │ (None, 426, 4)    │          0 │ conv1d_52[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_53 (Conv1D)  │ (None, 426, 1)    │          5 │ dropout_88[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 426, 1)    │          2 │ conv1d_53[0][0]   │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_53 (Add)        │ (None, 426, 1)    │          0 │ layer_normalizat… │
│                     │                   │            │ add_52[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 426, 1)    │      3,585 │ add_53[0][0],     │
│ (MultiHeadAttentio… │                   │            │ add_53[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_90          │ (None, 426, 1)    │          0 │ multi_head_atten… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 426, 1)    │          2 │ dropout_90[0][0]  │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_54 (Add)        │ (None, 426, 1)    │          0 │ layer_normalizat… │
│                     │                   │            │ add_53[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_54 (Conv1D)  │ (None, 426, 4)    │          8 │ add_54[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_91          │ (None, 426, 4)    │          0 │ conv1d_54[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_55 (Conv1D)  │ (None, 426, 1)    │          5 │ dropout_91[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 426, 1)    │          2 │ conv1d_55[0][0]   │
│ (LayerNormalizatio… │                   │            │                 

 Total params: 14,666 (57.29 KB)

 Trainable params: 14,666 (57.29 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step - loss: 0.6932 - sparse_categorical_accuracy: 0.5346 - val_loss: 0.6932 - val_sparse_categorical_accuracy: 0.5000
Epoch 2/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 17s 159ms/step - loss: 0.6932 - sparse_categorical_accuracy: 0.4970 - val_loss: 0.6931 - val_sparse_categorical_accuracy: 0.5000
Epoch 3/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 170ms/step - loss: 0.6932 - sparse_categorical_accuracy: 0.4939 - val_loss: 0.6931 - val_sparse_categorical_accuracy: 0.5000
Epoch 4/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 159ms/step - loss: 0.6932 - sparse_categorical_accuracy: 0.4799 - val_loss: 0.6931 - val_sparse_categorical_accuracy: 0.5000
Epoch 5/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 170ms/step - loss: 0.6931 - sparse_categorical_accuracy: 0.5026 - val_loss: 0.6931 - val_sparse_categorical_accuracy: 0.5000
Epoch 6/150
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 171ms/step - loss: 0.6932 - sparse_categorical_accuracy: 0.4982 - val_loss: 0.6931 - val_sparse_categorical_accuracy: 0.5000

[0.6931858658790588, 0.447826087474823]

## Conclusions

In about 110-120 epochs (25s each on Colab), the model reaches a training
accuracy of ~0.95, validation accuracy of ~84 and a testing
accuracy of ~85, without hyperparameter tuning. And that is for a model
with less than 100k parameters. Of course, parameter count and accuracy could be
improved by a hyperparameter search and a more sophisticated learning rate
schedule, or a different optimizer.

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

predictions = model.predict(x_test)
y_pred = np.argmax(predictions, axis=1)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

72/72 ━━━━━━━━━━━━━━━━━━━━ 6s 60ms/step
Accuracy: 0.5865
F1 Score: 0.5840
